In [1]:
import numpy as np
import pickle

In [2]:
def get_active_cells(deconvolved_traces):

    # going to return a dictionary with only active cells, formatted exactly the same as traces

    d = dict.fromkeys(deconvolved_traces.keys())

    for cell in deconvolved_traces:
        if deconvolved_traces[cell]['active'] == True:
            d[cell] = deconvolved_traces[cell]
        else:
            d.pop(cell,None)

    return d

In [3]:
BASE_PATH_1 = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/ID473/psilo/TSeries-10132023-1324-139/suite2p/plane0/"
BASE_PATH_2 = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/ID473/psilo/TSeries-10132023-1324-140/suite2p/plane0/"
file_path = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/ID473/psilo/TSeries-10132023-1324-140/suite2p/plane0/roimatch_139_140.csv"

iscellpath_1 = BASE_PATH_1 + "iscell.npy"
tracepath_1 = BASE_PATH_1 + "F.npy"
cell_dict_1 = BASE_PATH_1 + "cells.pkl"

iscellpath_2 = BASE_PATH_2 + "iscell.npy"
tracepath_2 = BASE_PATH_2 + "F.npy"
cell_dict_2 = BASE_PATH_2 + "cells.pkl"


iscell_1 = np.load(iscellpath_1)
traces_1 = np.load(tracepath_1)

iscell_2 = np.load(iscellpath_2)
traces_2 = np.load(tracepath_2)


with open(cell_dict_1, 'rb') as f:
    cell_dict_1 = pickle.load(f)

with open(cell_dict_2, 'rb') as f:
    cell_dict_2 = pickle.load(f)



In [4]:
ROI_indices_1 = (iscell_1[:,0] == 1).nonzero()
ROI_indices_1 = ROI_indices_1[0] # extracting the first part of the tuple
cell_IDs_1 = ROI_indices_1 + 1 # add 1 so we don't have zero indexing

# print(cell_IDs_1[436])

ROI_indices_2 = (iscell_2[:,0] == 1).nonzero()
ROI_indices_2 = ROI_indices_2[0] # extracting the first part of the tuple
cell_IDs_2 = ROI_indices_2 + 1 # add 1 so we don't have zero indexing

# print(cell_IDs_2[255])

In [5]:


# Load the CSV file into a NumPy array
matched_cells = np.genfromtxt(file_path, delimiter=',', skip_header=0)
matched_cells = matched_cells.astype(int) - 1  # Subtract 1 and convert to int in one step

# Extract columns 0 and 1
column_0 = matched_cells[:, 0]
column_1 = matched_cells[:, 1]

# Retrieve lists using NumPy indexing
cell_array = np.zeros_like(matched_cells)
cell_array[:,0] = cell_IDs_1[column_0]
cell_array[:,1] = cell_IDs_2[column_1]

print("Detected", len(cell_array[:,0]), "matched cells in first and",len(cell_array[:,1]), "in second recording (should be the same)")


active_1 = get_active_cells(cell_dict_1)

matched_active_1 = [cell for cell in cell_array[:,0] if cell in active_1.keys()]

print(len(matched_active_1), "active cells detected in pre- recording")

active_2 = get_active_cells(cell_dict_2)

matched_active_2 = [cell for cell in cell_array[:,1] if cell in active_2.keys()]

print(len(matched_active_2), "active cells detected in post- recording")


Detected 252 matched cells in first and 252 in second recording (should be the same)
99 active cells detected in pre- recording
99 active cells detected in post- recording


In [6]:
#  Make a two column array containing each cell and it's matched cell. 

coactive_cells = []
responded_in_minimum_one = []
became_inactive = []
became_active = []


for i, (cell_1, cell_2) in enumerate(cell_array):
    if cell_1 in matched_active_1 and cell_2 in matched_active_2:
        coactive_cells.append(cell_array[i, :])
    elif cell_1 in matched_active_1 and cell_2 not in matched_active_2:
        became_inactive.append(cell_array[i,0])
    elif cell_1 not in matched_active_1 and cell_2 in matched_active_2:
        became_active.append(cell_array[i,0])
    if cell_1 in matched_active_1 or cell_2 in matched_active_2:
        responded_in_minimum_one.append(cell_array[i, :])

# Convert the list of coactive_cells to a NumPy array
coactive_cells_array = np.array(coactive_cells)
active_only_first_array = np.array(became_inactive)
active_only_second_array = np.array(became_active)
active_in_minimum_one_array = np.array(responded_in_minimum_one)
print("There were", len(coactive_cells_array),"cells responsive in both recordings")
print("There were", len(became_inactive),"cells responsive only in the first recording")
print("There were", len(became_active),"cells responsive only in the second recording")
print("There were", len(responded_in_minimum_one),"cells responsive in at least one recording")

np.save(BASE_PATH_2 + "coactive_cells_array",coactive_cells_array)
np.save(BASE_PATH_2 + "active_only_first",active_only_first_array)
np.save(BASE_PATH_2 + "active_only_second",active_only_first_array)
np.save(BASE_PATH_2 + "matched_cells",cell_array)



There were 55 cells responsive in both recordings
There were 44 cells responsive only in the first recording
There were 44 cells responsive only in the second recording
There were 143 cells responsive in at least one recording


In [7]:
print(coactive_cells_array)

[[   6   24]
 [  14   11]
 [  19    9]
 [  22   50]
 [  37   70]
 [  38   61]
 [  42   89]
 [  50   37]
 [  53   64]
 [  56   79]
 [  59   68]
 [  78   53]
 [  81  120]
 [  91   88]
 [  92   34]
 [  94   74]
 [  95  189]
 [  97  598]
 [ 117  374]
 [ 129  230]
 [ 132   84]
 [ 133  123]
 [ 136  178]
 [ 139  481]
 [ 143  136]
 [ 154  143]
 [ 170   30]
 [ 172  190]
 [ 182  126]
 [ 187  257]
 [ 198  107]
 [ 212  222]
 [ 216  278]
 [ 220  380]
 [ 226  812]
 [ 271  554]
 [ 281  579]
 [ 284  477]
 [ 311 1134]
 [ 320  375]
 [ 327   39]
 [ 354  137]
 [ 372  452]
 [ 453  292]
 [ 496  309]
 [ 503  102]
 [ 518  366]
 [ 535  408]
 [ 554  655]
 [ 599 1033]
 [ 654  484]
 [ 773  273]
 [ 866  180]
 [1069  290]
 [1651 1052]]


In [8]:
print(matched_active_1)
print(matched_active_2)

[6, 14, 18, 19, 20, 21, 22, 25, 26, 32, 34, 35, 37, 38, 42, 48, 50, 53, 56, 59, 64, 72, 74, 78, 81, 83, 90, 91, 92, 93, 94, 95, 97, 112, 117, 119, 129, 132, 133, 136, 139, 140, 141, 143, 146, 154, 155, 170, 172, 174, 182, 187, 192, 197, 198, 207, 212, 216, 220, 226, 231, 263, 265, 271, 281, 284, 285, 289, 293, 311, 316, 320, 327, 335, 354, 372, 374, 376, 416, 453, 481, 486, 496, 501, 503, 518, 529, 535, 554, 599, 654, 753, 756, 765, 773, 866, 1069, 1367, 1651]
[16, 1, 24, 5, 43, 11, 9, 50, 70, 61, 89, 132, 37, 64, 79, 68, 156, 98, 177, 526, 378, 13, 66, 53, 18, 60, 120, 81, 88, 34, 74, 189, 598, 374, 386, 135, 230, 84, 123, 178, 481, 136, 410, 143, 105, 30, 190, 77, 125, 126, 257, 150, 107, 295, 181, 222, 325, 254, 278, 380, 812, 830, 590, 554, 579, 477, 448, 1134, 375, 39, 538, 259, 137, 237, 452, 719, 383, 508, 451, 679, 284, 292, 303, 41, 309, 102, 366, 408, 655, 186, 1033, 314, 931, 484, 273, 180, 290, 1318, 1052]


In [9]:
# Dictionary Additions:

#  Add a section to the post dictionary entitled 'matches' containing the array of matched cells.  
# Don't need more than this as you can work everything out from these. 

for cell_1, cell_2 in zip(cell_dict_1.keys(), cell_dict_2.keys()):
    cell_dict_1[cell_1]['matched_cells'] = cell_array
    cell_dict_2[cell_2]['matched_cells'] = cell_array

with open(BASE_PATH_1+'cells.pkl','wb') as f:
        pickle.dump(cell_dict_1,f)

with open(BASE_PATH_2+'cells.pkl','wb') as f:
        pickle.dump(cell_dict_2,f)
